# Points of interest

Open Street Maps is a great source of data for geo analytics, giving features such as amenties, transport locations and land use. In this notebook, we demonstrate how to use the overpass, nominatim apis to fetch information about supermarkets and a few other relevant POIs in london. 

As a toy problem, if you are in charge of finding new locations to open a waitrose supermarket, how would you go about it?
- we have information on demographics from the previous notebook on census2021
- we have a raster grid of 100m x 100m for london from the previous notebook
- we know where the current waitrose supermarkets are
- we can find out where our competitiors are
- we can find out where the transport stops are

We can start off with some simple rules to generate a score
- we can find out which locations are close to e.g. bus stop/tube stop (likely to have more business if people can reach it more easily)
- it would be desirable not to be too close to another waitrose or another supermarket (debatable, since there are times where opening next to similar shops do have advantages, but let's keep it simple)
- we would want to open where people are reasonably well off (waitrose is a premium supermarket after all...)
- we want to open somewhere there are a lot of people

I

In [1]:
from OSMPythonTools.overpass import Overpass,overpassQueryBuilder
from OSMPythonTools.api import Api

###  Query the nominatim API

We need to get the correct id for London to use for overpass queries. The nomainatim API provides this, as well as other info such as lat/lng, bounding box and so on. However, you do want to inspect the returned results manually if possible since there can be ambiguity around place names and the first result is not always what you want. 

In [2]:
from OSMPythonTools.nominatim import Nominatim
import pandas as pd
nominatim = Nominatim()
area_id = nominatim.query('London, UK').areaId()
print(area_id)

3600065606


In [3]:

nominatim.query('London, UK').toJSON()[:3]

[{'place_id': 344385499,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 65606,
  'boundingbox': ['51.2867601', '51.6918741', '-0.5103751', '0.3340155'],
  'lat': '51.5073219',
  'lon': '-0.1276474',
  'display_name': 'London, Greater London, England, United Kingdom',
  'class': 'place',
  'type': 'city',
  'importance': 0.9407827616237295,
  'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_place_city.p.20.png'},
 {'place_id': 297653298,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 51800,
  'boundingbox': ['51.5068696', '51.5233122', '-0.1138295', '-0.072747'],
  'lat': '51.5156177',
  'lon': '-0.0919983',
  'display_name': 'City of London, Greater London, England, United Kingdom',
  'class': 'boundary',
  'type': 'administrative',
  'importance': 0.6865111547516773,
  'icon': 'https://nominatim.openstreetmap.org/ui/map

Using the areaid, we can now use overpass to query for the relevant data

In [4]:
overpass = Overpass()
query = overpassQueryBuilder(area=area_id, elementType='nwr', selector='"shop"="supermarket"', out='center')
supermarkets= overpass.query(query)

Note there are two types of entries that can be returned-- nodes, which have a lat/lng keys, and ways, which have a 'center' key, corresponding to a dict with lat/lng value of the center of the way. 

For example

Node:
```python
{'type': 'node',
 'id': 25744394,
 'lat': 51.5620778,
 'lon': -0.1493587,
 'tags': {'brand': 'Tesco Express',
  'brand:wikidata': 'Q98456772',
  'brand:wikipedia': 'en:Tesco',
  'contact:website': 'https://www.tesco.com/store-locator/london/23-25-swains-ln',
  'name': 'Tesco Express',
  'shop': 'supermarket',
  'wheelchair': 'limited'}}

```

Way:
```python
{'type': 'way',
  'id': 137459762,
  'center': {'lat': 51.4642954, 'lon': 0.1074691},
  'nodes': [1507739897,
   1507739898,
   1507739899,
   1507739900,
   1507739901,
   1507739902,
   1507739897],
  'tags': {'addr:country': 'GB',
   'addr:housenumber': '46-60',
   'addr:postcode': 'DA16 3HF',
   'addr:street': 'Upper Wickham Lane',
   'brand': 'Lidl',
   'brand:wikidata': 'Q151954',
   'brand:wikipedia': 'en:Lidl',
   'building': 'yes',
   'designation': 'Welling',
   'fhrs:id': '1009721',
   'name': 'Lidl',
   'operator': 'Lidl',
   'shop': 'supermarket',
   'source': 'Bing'}}
```

**extracting information from the returned results**

In [5]:
# calling on the `elements()` method like so gives you entries in the form of json
supermarkets.elements()[0]._json

{'type': 'node',
 'id': 25744394,
 'lat': 51.5620778,
 'lon': -0.1493587,
 'tags': {'brand': 'Tesco Express',
  'brand:wikidata': 'Q98456772',
  'brand:wikipedia': 'en:Tesco',
  'contact:website': 'https://www.tesco.com/store-locator/london/23-25-swains-ln',
  'name': 'Tesco Express',
  'shop': 'supermarket',
  'wheelchair': 'limited'}}

In [14]:
supermarkets_df = []
for x in  supermarkets.elements():
    entry = x._json
    if entry['type'] == 'way':
        supermarkets_df.append(dict(
            id = entry['id'],
            lat=entry.get('center', {}).get('lat'), 
            lng = entry.get('center', {}).get('lon'), 
            brand=entry.get('tags', {}).get('brand:wikipedia'), 
            name=entry.get('tags', {}).get('name')
        ))
    elif entry['type'] == 'node':
        supermarkets_df.append(dict(
            id = entry['id'],
            lat=entry.get('lat'), 
            lng = entry.get('lon'), 
            brand=entry.get('tags', {}).get('brand:wikipedia'), 
            name=entry.get('tags', {}).get('name')
        ))
supermarkets_df = pd.DataFrame(supermarkets_df)


In [15]:
supermarkets_df.sample(frac=1.).head(10)

,id,lat,lng,brand,name
350,5842529172,51.426015,-0.051829,None,PFC
467,8252199923,51.469922,-0.358027,None,Safa & Marna Supermarket
1010,444642774,51.574904,-0.012929,None,Iceland
1035,527096586,51.368451,-0.196835,en:Sainsbury's,Sainsbury's
413,6798574590,51.550019,-0.191182,None,Sainsbury's Local
355,5892620652,51.408780,-0.105924,None,The Co-operative Food
801,166129315,51.607240,-0.209046,en:Waitrose & Partners,Waitrose
791,158514106,51.461533,-0.301876,en:Waitrose & Partners,Waitrose
32,313617786,51.493274,-0.239557,None,Thai Smile
87,677228899,51.563595,-0.355175,None,Iceland


In [18]:
supermarkets_df.name.value_counts().head(10)

Lidl             108
Sainsbury's       97
Co-op Food        66
Waitrose          59
Tesco Express     45
Asda              37
Tesco             35
ALDI              34
Morrisons         28
Co-op             24
Name: name, dtype: int64